## **The Sparks Foundation - GRIP 03/2021**

## Author: DIVYA KUNDALA



# GLOBAL TERRORISM

## Importing necessary libraries

In [ ]:
# Importing all the libraries needed
import math
import warnings
import numpy as np 
import pandas as pd 
from pandas import DataFrame
import plotly.express as px
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from wordcloud import WordCloud
from scipy import signal
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
from mpl_toolkits.basemap import Basemap
import matplotlib.animation as animation
warnings.filterwarnings('ignore')
import pydotplus
from io import StringIO
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
import time
import graphviz
from graphviz import Source
from sklearn import tree
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from tabulate import tabulate
from matplotlib.font_manager import FontProperties

## 1. Loading and Pre-Processing the Data

In [ ]:
global_data = pd.read_csv("globalterrorismdb_0718dist.csv",encoding='ISO-8859-1')
global_data.head() #listing the first 5 rows

In [ ]:
global_data.shape #check the shape of df

In [ ]:
global_data.info()

In [ ]:
global_data.columns

In [ ]:
# to check if there is duplicated entry in the dataset
global_data.duplicated().sum()

In [ ]:
#Renaming the columns
global_data.rename(columns={'iyear':'Year','imonth':'Month','iday':'Day','country_txt':'Country','provstate':'State',
                   'region_txt':'Region','city':'City','latitude':'Latitude','longitude':'Longitude',
                   'attacktype1_txt':'AttackType','targtype1_txt':'TargetType','targsubtype1_txt':'TargetSubType',
                   'weaptype1_txt':'WeaponType','weapsubtype1_txt':'WeaponSubType','gname':'Group','nkill':'Killed','nwound':'Wounded','summary':'Summary','motive':'Motive', 'success':'Success','weapdetail':'Detail','propextent_txt':'Severity'},inplace=True)

In [ ]:
#displaying the columns which are necessary for our project

global_data=global_data[['eventid','Year','Month','Day','Country','State','Region','City','Latitude','Longitude','AttackType','TargetType','TargetSubType'
       ,'suicide','Killed','Wounded','Summary','Group','WeaponType','WeaponSubType','Motive', 'Success', 'Detail','Severity']]

global_data.head()

In [ ]:
# Checking for the null values
global_data.isnull().sum()

In [ ]:
#filling the NA values
global_data["State"].fillna("Unknown", inplace = True)
global_data["City"].fillna("Unknown", inplace = True)
global_data["Latitude"].fillna(global_data["Latitude"].mean(), inplace = True)
global_data["Longitude"].fillna(global_data["Longitude"].mean(), inplace = True)
global_data["TargetSubType"].fillna("Unknown", inplace = True)
global_data["Summary"].fillna("Unknown", inplace = True)
global_data["Killed"].fillna(0, inplace = True)
global_data["Wounded"].fillna(0, inplace = True)
global_data["WeaponSubType"].fillna("Unknown", inplace = True)
global_data["Motive"].fillna("Unknown", inplace = True)
global_data["Severity"].fillna("Unknown", inplace = True)
global_data["Detail"].fillna("Unknown", inplace = True)

In [ ]:
#shortening few words in the dataset
global_data['Country'] = global_data['Country'].replace('South Vietnam','Vietnam', regex=True)
global_data['Weaptype'] = global_data['WeaponType'].replace('Vehicle .*','Vehicle', regex=True)
global_data['AttackType'] = global_data['AttackType'].replace('Hostage Taking .*','Hostage Taking', regex=True)

In [ ]:
global_data.describe()

In [ ]:
global_data.shape

In [ ]:
global_data.info()

## 2. Data Visualisation

### Destructive Data Analysis

In [ ]:
#tabulating the highest destruction corresponding to the parameters
global_List = [['Country', global_data['Country'].value_counts().idxmax(), max(global_data['Country'].value_counts())],['City', global_data['City'].value_counts().index[1], global_data['City'].value_counts()[1]],['Region', global_data['Region'].value_counts().idxmax(), max(global_data['Region'].value_counts())],['Year', global_data['Year'].value_counts().idxmax(), max(global_data['Year'].value_counts())],['Month', global_data['Month'].value_counts().idxmax(), max(global_data['Month'].value_counts())],['Group', global_data['Group'].value_counts().index[1], global_data['Group'].value_counts()[1]],['Most Attack Types', global_data['AttackType'].value_counts().idxmax(), max(global_data['AttackType'].value_counts())]]
dist_data = DataFrame (global_List,columns=['Parameters','Highest Destruction','Attacks_count'])
print (dist_data)

In [ ]:
#Wordcloud dipicting the states that are affected with the terrorists attack
St = global_data.State.dropna(False)
plt.subplots(figsize=(20,10))
wordcloud = WordCloud(background_color = 'green',
                     width = 500,
                     height = 400).generate(' '.join(St))
plt.axis('off')
plt.imshow(wordcloud)
plt.show()

In [ ]:
#Plotting heatmap 
plt.figure(figsize=(10,5))
sns.heatmap(global_data.corr(), annot=True)

## 3. Explaratory Data Analysis
###     No. of global attacks per year

In [ ]:
#To plot the graph to visualise the no. of incidents occured since 1970
plt.figure(figsize=(15,10))
ax = sns.countplot(x='Year',data=global_data,palette='mako')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right")
plt.title('Global Terrorist Attack(1970-2017)',size=15)
plt.ylabel('Incidents')

In [ ]:
#Interactive graph showing the details on hover
plotting=global_data.Year.value_counts()
fig = px.bar(plotting, x=plotting.index, y=plotting.values, title="No. of terrorist attacks", labels={'y':'No. of Attacks', 'index':'Year'})
fig.show()             

### Terrorist Attack in each country

In [ ]:
# to plot the terrorst attack on top 10 Country
plt.figure(figsize=(15,10))
sns.barplot(x = global_data['Country'].value_counts().values[:10], y = global_data['Country'].value_counts()[:10].index,palette = 'rocket')
plt.xlabel('Number of Attacks')
plt.ylabel('Countries')
plt.title('Top-10 Countries with terrorist attacks',size=15)

### Terrorist Attack in states

In [ ]:
# plotting the terrorist attacks in top 10 states
fig = plt.figure(figsize=(15,6))
sns.barplot(global_data['State'].value_counts()[:10].index,global_data['State'].value_counts()[:10].values,palette='rocket')
plt.title('Top 10 States Affected')
plt.xlabel('States')
plt.ylabel('No. of attacks')


### Terrorist attacks in Region

In [ ]:
#top 10 regions with terror attacks
fig = plt.figure(figsize=(10,6))
sns.barplot(global_data['Region'].value_counts()[:10].index,global_data['Region'].value_counts()[:10].values)
plt.title('Top 10 Affected Regions')
plt.xlabel('Region')
plt.ylabel('Counts')
plt.xticks(rotation=90); #No-overlapping of x labels

### Casualities by region

In [ ]:
gtd_region_wise = global_data[['Region','Killed','Wounded']].copy()
gtd_region_wise = gtd_region_wise.groupby(by=['Region']).sum()
gtd_region_wise

### Groups with most terrorist attacks

In [ ]:
#terrorist groups
fig = plt.figure(figsize=(15,6))
sns.barplot(global_data['Group'].value_counts()[:10].index,global_data['Group'].value_counts()[:10].values,palette='rocket')
plt.title('Group with Most Terrorist Attacks')
plt.xlabel('Group')
plt.ylabel('Counts')
#Rotate the labels on X-axis so that they don't overlap
plt.xticks(rotation=90);

In [ ]:
#Group attacked corresponding to the Country
dfClass1 = global_data.loc[global_data.Year.isin(np.array(range(1970,2017)))]
fig = px.sunburst(dfClass1, path=['Group','Country'],
                 labels={"count": "Attacks",'id':'path'},
                 title='Group Name vs Country attacked')
fig.show()

### Targets for the attack

In [ ]:
#To plot the terrorists favorite places of attack
plt.figure(figsize=(15,10))
sns.countplot(global_data['TargetType'],order=global_data['TargetType'].value_counts().index,palette='autumn')
plt.xticks(rotation=90)
plt.xlabel('Type')
plt.title('Targets for the attack',size=15)

In [ ]:
#plot target type corresponding to the Country
fig = px.sunburst(dfClass1, path=['TargetType','Country'],
                 labels={"count": "Attacks",'id':'path'},
                 title='Target type vs Country')
fig.show()

### Terrorist Attack Types

In [ ]:
graph_data =global_data['AttackType'].apply(lambda x: x if x in ['Bombing/Explosion','Armed Assault','Assassination',
                                                            'Hostage Taking',
                                                            'Facility/Infrastructure Attack'] else 'Others')
attack_type = graph_data.value_counts().tolist()

labels = ['Bombing/Explosion','Armed Assault','Assassination','Hostage Taking','Facility/Infrastructure Attack','Others']
print("The no.of attacks: ", labels, attack_type)

sizes = []

for i in attack_type:
    percent = i*100/len(global_data['AttackType'])
    sizes.append(percent)

fig, ax = plt.subplots(figsize=(10,10))
patches, texts, autotexts = ax.pie(sizes, labels=labels, autopct='%1.1f%%',
        startangle = -20, shadow = True,
        explode = (0.05, 0, 0, 0, 0, 0),
        colors = sns.color_palette("Paired", 8)[:5]+[(0.5843137254901961, 0.6470588235294118, 0.6509803921568628)],
        textprops={'fontsize':15,'weight':'light','color':'k'})

ax.axis('equal')
plt.title('Terrorist Attack Types', fontsize= 25, pad= -70, weight ='bold', 
             color = sns.cubehelix_palette(8, start=.5, rot=-.75)[-3]) #pad change the distance from title to graph
plt.tight_layout()
ax.legend(loc='lower right',framealpha = 0.5,bbox_to_anchor=(1.8,0.5,0.1,1), prop={'size': 14})
fig.show()

In [ ]:
#Attack type, weapon type against the Country
weapon_used = global_data[["eventid","Country",'Year', 'Month', 'Day','WeaponType','AttackType']]
fig = px.sunburst(weapon_used, path=['AttackType','WeaponType','Country'],
                 labels={"count": "Attacks",'id':'path'},
                 title='Attack Type - Weapon type - Country')
fig.show()

In [ ]:
#Checking the weapon type chosen for attack
plt.figure(figsize=(20, 10))
sns.countplot(x="AttackType", hue="WeaponType", data=global_data) 
plt.show()

In [ ]:
# Total Number of people killed in terror attack
killed_data = global_data.loc[:,'Killed']
print('Number of people killed by terror attack:', int(sum(killed_data.dropna())))# drop the NaN values
# Let's look at what types of attacks these deaths were made of.
attack_data = global_data.loc[:,'AttackType']
# attackData
typeKill_data = pd.concat([attack_data, killed_data], axis=1)
#count of killed against the attack type
typeKill_FormatData = typeKill_data.pivot_table(columns='AttackType', values='Killed', aggfunc='sum')
typeKill_FormatData

In [ ]:
#Attack type vs Region

pd.crosstab(global_data.Region, global_data.AttackType).plot.barh(stacked=True,width=1, color=sns.color_palette('magma',9))
fig=plt.gcf()
fig.set_size_inches(12,8)
plt.show()

In [ ]:
#To check the success rate of attacks
success = len(global_data[global_data["Success"]==1])
failed =  len(global_data[global_data["Success"]==0])
list_sf = [success, failed]
plt.pie(list_sf, labels = ["Success", "Failed"], autopct = "%.1f%%", startangle = 90, explode=[0.1,0])
plt.title("Success vs Failure rate of attacks", fontsize=16)
plt.show()

In [ ]:
# plot showing the attacks,killed and wounded in each year
world_killed = global_data.groupby(["Year"])["Killed"].sum()
world_wounded = global_data.groupby(["Year"])["Wounded"].sum()
fig= go.Figure(data=[go.Bar(name="Attacks", x=plotting.index, y=plotting.values),go.Bar(name="Killed", x=world_killed.index, y=world_killed.values), go.Bar(name="Wounded", x=world_wounded.index,y=world_wounded.values),])
fig.update_layout(title="Total Attacks vs No. of people killed/wounded per year")
fig.show()

In [ ]:
# plotting attack type Vs No. of people killed and wounded

attack_type_kill = global_data[['Killed','Wounded']].groupby(global_data['AttackType']).sum()
attack_type_kill.plot(kind='bar')
plt.title("Attack type vs no. of people killed and wounded")
plt.xlabel("AttackType")
plt.ylabel("No. of people killed/wounded")

In [ ]:
# plotting the attacks in the map
px.scatter_mapbox(global_data, lat="Latitude", lon="Longitude", color="AttackType",size="Killed",hover_name="City",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10,
                  mapbox_style="carto-positron",title="Attacks in each city")

In [ ]:
global_Map=global_data.where(global_data['Latitude']!=None).sort_values(by='Year')

global_Map=global_Map.groupby(['Year','Latitude','Longitude','City','Country']).count()
a=global_Map.index.values
l=list(a)
Alt = [x[1] for x in l]
Long = [y[2] for y in l]
Years= [a[0] for a in l]
Countries= [c[-1] for c in l]
Cities = [c[-2] for c in l]
#print(Cities)
Alt = np.array(Alt)
Long = np.array(Long)
Years = np.array(Years)
Countries = np.array(Countries)
Cities=np.array(Cities)
fig = px.scatter_geo(global_Map, lon=Long,lat=Alt,hover_name=Cities,animation_frame=Years,projection="equirectangular",
                    labels={"eventid": "Attacks",'animation_frame':'Year'})
fig.update_layout(title=' Scatter plot: Map of terrorism attacks in the world from 1970 to 2017')
fig.show()

In [ ]:
#terror attacks in city
filtered_Year = global_data['Year'] == 2014
filtered_Data = global_data[filtered_Year] # filter data
map_data = filtered_Data.loc[:,'City':'Longitude'] #city and longitude
map_data = map_data.dropna() # drop NaN values for plotting
map_data1 = map_data.values.tolist()

In [ ]:
map = folium.Map(location = [0, 30], tiles='CartoDB positron', zoom_start=2)
# clustered marker
markerCluster = folium.plugins.MarkerCluster().add_to(map)
for point in range(0, len(map_data1)):
    folium.Marker(location=[map_data1[point][1],map_data1[point][2]],popup = map_data1[point][0]).add_to(markerCluster)

In [ ]:
map

In [ ]:
terror_df_group = global_data.dropna(subset=['Latitude','Longitude'])
terror_df_group = terror_df_group.drop_duplicates(subset=['City','Group'])
terrorist_groups = global_data.Group.value_counts()[1:8].index.tolist()
terror_df_group = terror_df_group.loc[terror_df_group.Group.isin(terrorist_groups)]

m1 = folium.Map(location=[20, 0], tiles="CartoDB positron", zoom_start=2)
marker_cluster = MarkerCluster(name='clustered icons',overlay=True,control=False,icon_create_function=None)
for i in range(0,len(terror_df_group)):
    marker=folium.Marker([terror_df_group.iloc[i]['Latitude'],terror_df_group.iloc[i]['Longitude']]) 
    popup='Group:{}<br>City:{}'.format(terror_df_group.iloc[i]['Group'],terror_df_group.iloc[i]['City'])
    folium.Popup(popup).add_to(marker)
    marker_cluster.add_child(marker)
marker_cluster.add_to(m1)
folium.TileLayer('openstreetmap').add_to(m1)
folium.TileLayer('cartodbdark_matter').add_to(m1)
folium.TileLayer('stamentoner').add_to(m1)
folium.LayerControl().add_to(m1)

m1

In [ ]:
#HeatMap for 2014 attacks
attack=global_data.loc[global_data.Year==2014][['Latitude','Longitude']]
attack.Latitude.fillna(0, inplace = True)
attack.Longitude.fillna(0, inplace = True) 
World=folium.Map(location=[0,0],zoom_start=2)
HeatMap(data=attack, radius=16).add_to(World)
print('Terrorism spread in 2014')
World

In [ ]:
#Basemap for attacks over years
plt.figure(figsize=(24,12))
map = Basemap(projection='hammer',lat_0=20,lon_0=-20) 
map.drawcoastlines(linewidth=0.25)
map.drawcountries(linewidth=0.25)
map.fillcontinents(color='yellow',lake_color='deepskyblue')

map.drawmapboundary(fill_color='deepskyblue')

# draw lat/lon grid lines every 30 degrees.
map.drawmeridians(np.arange(0,360,30))
map.drawparallels(np.arange(-90,90,30))

lons=global_data['Longitude']
lats=global_data['Latitude']
long = lons.tolist()
lati = lats.tolist()
x,y = map(long, lati)
map.plot(x, y, 'ro', markersize=.05)
plt.title("Global Terrorist Attacks (1970 - 2016)")

plt.show()

In [ ]:
#Attack in south Asia
reg_groupby = global_data.groupby(["Region"])
gd_df = reg_groupby.get_group("South Asia")
gd_df['Affected'] = gd_df['Killed'] + gd_df['Wounded']
# boundary of south asia (lat and lon coordinates)
llon = 58.8
ulon = 98.9
llat = 5.04
ulat = 41.06
years=list(gd_df.Year.unique())
fig = plt.figure(figsize=(14, 7), dpi=150)

def year_attack(Year):
    plt.clf() # clear the previous plot 
    plt.title('Terrorism In South Asia '+'\n'+'Year:' +str(Year))
    my_map = Basemap(projection='merc',resolution='l', llcrnrlon=llon, llcrnrlat=llat-1, urcrnrlon=ulon, urcrnrlat=ulat+3, area_thresh=100, ) # low resolution
    lat_gif=list(gd_df[gd_df['Year']==Year].Latitude)
    long_gif=list(gd_df[gd_df['Year']==Year].Longitude)
    x_gif,y_gif=my_map(long_gif, lat_gif)
    my_map.scatter(x_gif, y_gif,s=[fat*1.10 for fat in gd_df[gd_df['Year']==Year].Affected], color = 'OrangeRed')
    my_map.drawcoastlines()
    my_map.drawcountries()
    my_map.shadedrelief()

year_ani_SA = animation.FuncAnimation(fig, year_attack, years, interval=1500)
year_ani_SA.save('year_attack_ani.gif', writer='pillow', fps=1, dpi=150)

## 4. Classification

In [ ]:
import pandas as pd
global_terrorism_df = pd.read_csv('globalterrorismdb_0718dist.csv',encoding='ISO-8859-1')
# To remove all columns having null values > 5000
for i in global_terrorism_df.columns:
    if global_terrorism_df[i].isnull().sum()>5000:
        global_terrorism_df.drop(columns=i, inplace=True)
#Fill the na values
global_terrorism_df['doubtterr'].fillna(-9, inplace=True)
global_terrorism_df['multiple'].fillna(0, inplace=True)
global_terrorism_df['natlty1'].fillna(0, inplace=True)
global_terrorism_df['ishostkid'].fillna(-9, inplace=True)

# quantizing dataset
global_terrorism_df['provstate'] = pd.factorize(global_terrorism_df['provstate'])[0]
global_terrorism_df['iyear'] = pd.factorize(global_terrorism_df['iyear'])[0]
global_terrorism_df['city'] = pd.factorize(global_terrorism_df['city'])[0]
global_terrorism_df['country'] = pd.factorize(global_terrorism_df['country'])[0]
global_terrorism_df['target1'] = pd.factorize(global_terrorism_df['target1'])[0]
global_terrorism_df['gname'] = pd.factorize(global_terrorism_df['gname'])[0]
global_terrorism_df['natlty1'] = pd.factorize(global_terrorism_df['natlty1'])[0]

attacktype_dict = {}
attacktype_code = list(global_terrorism_df.attacktype1.unique())
attacktype_name = list(global_terrorism_df.attacktype1_txt.unique())
for i in range(len(attacktype_code)):
    attacktype_dict[attacktype_code[i]] = attacktype_name[i]
# further reduction of unnecessary columns after looking at the Codebook (documentation)
global_terrorism_df.drop(columns=['eventid', 'extended', 'country_txt', 'region_txt', 'specificity', 'vicinity',
                 'crit1', 'crit2', 'crit3', 'attacktype1_txt', 'targtype1_txt', 'natlty1_txt',
                 'guncertain1', 'individual', 'weaptype1_txt', 'dbsource', 'INT_MISC'], inplace=True)    
#To check the null values if any
global_terrorism_df.isna().any()

In [ ]:
global_terrorism_df=global_terrorism_df.dropna()

In [ ]:
global_terrorism_df.isna().any()

In [ ]:
global_terrorism_df.info()

In [ ]:
X = global_terrorism_df.drop(['attacktype1'], axis=1, inplace=False)
y = global_terrorism_df['attacktype1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=44, shuffle =True)
print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)

In [ ]:
#GBC model
start_time = time.time()
GBCModel = GradientBoostingClassifier(n_estimators=100,max_depth=3,random_state=33) 
GBCModel.fit(X_train, y_train)
predictions = GBCModel.predict(X_test)
acc_score_gbc = accuracy_score(predictions, y_test)
print("Accuracy score of GBC: ", round(accuracy_score(predictions, y_test),2))
time_gbc=time.time() - start_time
print("Time taken by GBC: ", round((time.time() - start_time),3),'seconds')

In [ ]:
# Random Forest Model
start_time = time.time()
model_RFC=RandomForestClassifier(n_estimators=100)
model_RFC.fit(X_train, y_train)
pred_rfc = model_RFC.predict(X_test)
acc_score_rfc = accuracy_score(y_test,pred_rfc)
print('Accuracy score of RFC: ', round(accuracy_score(y_test,pred_rfc),2))
time_rfc=time.time() - start_time
print("Time taken by RFC: ", round((time.time() - start_time),3),'seconds')

In [ ]:
#Gaussaian NB
start_time = time.time()
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred  =  classifier.predict(X_test)
acc_score_gb = accuracy_score(y_test,y_pred)
print('Accuracy score of GaussianNB: ', round(accuracy_score(y_test,y_pred ),2))
time_gb=time.time() - start_time
print("Time taken by GaussianNB: ", round((time.time() - start_time),3),'seconds')

In [ ]:
# AdaBoost Classifier
start_time = time.time()
abc = AdaBoostClassifier(n_estimators=100, random_state=0)
y_pred_abc = abc.fit(X_train, y_train).predict(X_test)
acc_score_abc = accuracy_score(y_test,y_pred_abc)
print("Accuracy of AdaBoost Classifier:", round(accuracy_score(y_test,y_pred_abc),2))
time_abc=time.time() - start_time
print("Time taken by AdaBoost classifier: ", round((time.time() - start_time),3),'seconds')

In [ ]:
#to graphically represent the accuracy of the models
data = {'GBC':acc_score_gbc, 'RFC':acc_score_rfc, 'GB':acc_score_gb, 'ABC':acc_score_abc}
model = list(data.keys())
score = list(data.values())
fig=go.Figure(data=[go.Bar(x=model, y=score)])
fig.update_layout(title="Accuracy scores of the models")
fig.show()

In [ ]:
#To compare the time takem by the models
time_data={'GBC':time_gbc, 'RFC':time_rfc, 'GB':time_gb, 'ABC':time_abc}
time=list(time_data.values())
fig=go.Figure(data=[go.Bar(x=model, y=time)])
fig.update_layout(title="timing taken by models")
fig.show()

In [ ]:
#Precision, recall and f score for the models
print('GBC\n\nPrecision: {:.2f}\nRecall: {:.2f}\nF-score: {:.2f}'.format(precision_score(y_test, predictions,average="macro"), recall_score(y_test, predictions,average="macro"), f1_score(y_test, predictions,average="macro")))
print('\nRFC\n\nPrecision: {:.2f}\nRecall: {:.2f}\nF-score: {:.2f}'.format(precision_score(y_test, pred_rfc,average="macro"), recall_score(y_test, pred_rfc,average="macro"), f1_score(y_test, pred_rfc,average="macro")))
print('\nGaussianNB\n\nPrecision: {:.2f}\nRecall: {:.2f}\nF-score: {:.2f}'.format(precision_score(y_test, y_pred,average="macro"), recall_score(y_test, y_pred,average="macro"), f1_score(y_test, y_pred,average="macro")))
print('\nABC\n\nPrecision: {:.2f}\nRecall: {:.2f}\nF-score: {:.2f}'.format(precision_score(y_test, y_pred_abc,average="macro"), recall_score(y_test, y_pred_abc,average="macro"), f1_score(y_test, y_pred_abc,average="macro")))

In [ ]:
#visualising the machine learning models of India with respect to the attack type
fontP = FontProperties()
fontP.set_size('small')
def plot_classifier_b(model):
    ax=plt.gca()
    temp_df = global_terrorism_df.country == 92
    terrorism_data_in_country = global_terrorism_df[temp_df]
    terrorism_data_in_country_X = terrorism_data_in_country.drop(['attacktype1'], axis=1, inplace=False)
    terrorism_data_in_country_y = terrorism_data_in_country['attacktype1']
  
    y_plot_pred = model.predict(terrorism_data_in_country_X)

    dict_attack_data = {'Attack_number': [1,2,3,4,5,6,7,8,9], 'Attack':['Assassination', 'Armed Assault', 'Bombing/Explosion', 'Hijacking', 'Hostage Taking (Barricade Incident)', 'Hostage Taking (Kidnapping)', 'Facility/Infrastructure Attack','Unarmed Assault', 'Unknown']}  

    attack_list_pred = []
 
    scatter = ax.scatter(terrorism_data_in_country_X.latitude, terrorism_data_in_country_X.longitude, c = y_plot_pred, cmap='rainbow')

    plt.colorbar(scatter)

    legend1 = ax.legend(*scatter.legend_elements(), loc='upper left', prop=fontP, bbox_to_anchor=(1.5, 1),
                     title="Attacks", ncol=4, fontsize=8)
  
    legend_df = pd.DataFrame.from_dict(dict_attack_data)
    print(tabulate(legend_df, headers = legend_df.columns))
  

plot_classifier_b(model_RFC)

In [ ]:
plot_classifier_b(classifier)

In [ ]:
plot_classifier_b(abc)

In [ ]:
plot_classifier_b(GBCModel)

### We can see that RFC gives the better accuracy score than any other models and it works well with sparse data sets with a high number of classes. 

In [ ]:
'''The following insights are drawn:

 
          Parameters         Highest Destruction  Attacks_count
0            Country                        Iraq          24636
1               City                     Baghdad           7589
2             Region  Middle East & North Africa          50474
3               Year                        2014          16903
4              Month                           5          16875
5              Group                     Taliban           7478
6  Most Attack Types           Bombing/Explosion          88255 '''



As we can see that the attack type which is mostly used is bombing/explosion. There should be a proper security measures to be implemented by the government to overcome this situation.

Next steps:
1.   The model does not consider groups that are unknown. I would improve this by factoring in classification confidence in the label and improved via feature engineering which may result in high accuracy.

2.    The latitude and longitude contain null values. The accuracy can be improved by filling the data. This would involve cross-referencing other geolocation packages.
